## <center>Processing Drone Imagery - Metashape Pro</center>

##### Import necessary modules

In [5]:
import Metashape
import os

##### Define input and output folders

In [2]:
image_folder = "S:/Zack/Imagery/Chestnut/Agisoft/2024/Wintergreen/AVNW/Raw Images/"
output_folder = "S:/Zack/Imagery/Chestnut/Agisoft/2024/Wintergreen/AVNW/Outputs/"

##### Create Metashape project

In [ ]:
doc = Metashape.Document()
doc.save(output_folder + "project.psx")

##### Scan input folder for images

In [ ]:
def find_files(folder, types):
    return [entry.path for entry in os.scandir(folder) if (entry.is_file() and os.path.splitext(entry.name)[1].upper() in types)]

# load photos from image folder
photos = find_files(image_folder, [".JPG", ".JPEG", ".TIF", ".TIFF"])

print(len(photos), "photos found")

##### Load images from folder

In [ ]:
chunk = doc.addChunk()
chunk.addPhotos(photos)

print(str(len(chunk.cameras)) + " images loaded")

doc.save()

##### Align the loaded images

In [ ]:
chunk.alignCameras(adaptive_fitting=False,
                   min_image=2)
doc.save()

##### Generate tie points

In [ ]:
chunk.matchPhotos(downscale=0, 
                  keypoint_limit=60000, 
                  tiepoint_limit=0, 
                  generic_preselection=True, 
                  reference_preselection=True,
                  filter_mask=True,
                  mask_tiepoints=True,
                  filter_stationary_points=True,
                  keep_keypoints=False,
                  reset_matches=False)
doc.save()

##### Realign matched images

In [ ]:
chunk.alignCameras(adaptive_fitting=False,
                   min_image=2)
doc.save()

##### Optimize cameras

In [ ]:
chunk.optimizeCameras(tiepoint_covariance=True,
                      fit_f=True,
                      fit_cx=True,
                      fit_cy=True,
                      fit_b1=True,
                      fit_b2=True,
                      fit_k1=True,
                      fit_k2=True,
                      fit_k3=True,
                      fit_k4=True,
                      fit_p1=True,
                      fit_p2=True,
                      fit_corrections=True,
                      adaptive_fitting=True)
doc.save()

In [ ]:
def grad_selection_RU(RU_thrsh, num_tries=4, pct_del=10, thrsh_incr=1):
    n=0
    target_thrsh = 10
    init_thrsh = RU_thrsh
    points = chunk.tie_points.points
    # compute number of starting points
    points_start_num = len(points)
    f = Metashape.TiePoints.Filter()
    f.init(chunk, criterion = Metashape.TiePoints.Filter.ReconstructionUncertainty)
    # select points that exceed initial threshold 
    f.selectPoints(init_thrsh)
    # count number of points selected
    nselected = len([True for point in points if point.valid is True and point.selected is True])
    # check to see if any points were selected
    pct_selected = (nselected / points_start_num)*100
    # decrease the initial threshold if few points were selected
    if pct_selected <= 1:
        while True:
            print(f"Current threshold is {init_thrsh}. Adjusting downward by {thrsh_incr}")
            init_thrsh -= thrsh_incr
            f.selectPoints(init_thrsh)
            nselected = len([True for point in points if point.valid is True and point.selected is True])
            ps = (nselected / points_start_num)*100
            if ps >= 1:
                break
    # increase the initial threshold if too many points were selected
    elif pct_selected > pct_del:
        while True:
            if pct_selected > pct_del:
                init_thrsh += thrsh_incr
                f.selectPoints(init_thrsh)
                nselected = len([True for point in points if point.valid is True and point.selected is True])
                ps = (nselected / points_start_num)*100
                if ps < pct_del:
                    break
    print(f"New adjusted initial threshold is {init_thrsh}. Beginning gradual selection.")
    # begin iterative gradual selection
    n+=1
    print(f"Removing {nselected} points.")
    f.removePoints(init_thrsh)
    init_thrsh -= thrsh_incr
    chunk.optimizeCameras(adaptive_fitting=True,fit_f = True, fit_cx = True, fit_cy = True, fit_b1 = True, fit_b2 = True, fit_k1 = True, fit_k2 = True, fit_k3 = True, fit_k4 = True, fit_p1 = True, fit_p2 = True)
    points = chunk.tie_points.points
    npoints = len(points)
    while True:
        n+=1
        if n>num_tries or init_thrsh<=target_thrsh or (100*((points_start_num-npoints)/points_start_num))>=50:
            break
        else:
            points = chunk.tie_points.points
            npoints = len(points)
            f.selectPoints(init_thrsh)
            nselected = len([True for point in points if point.valid is True and point.selected is True])
            pct_selected = (nselected / npoints)*100
            while True:
                if pct_selected <= pct_del:
                    init_thrsh -= thrsh_incr/5
                    f.selectPoints(init_thrsh)
                    nselected = len([True for point in points if point.valid is True and point.selected is True])
                    pct_selected = (nselected/npoints)*100
                else:
                    break
            f.selectPoints(init_thrsh)
            nselected = len([True for point in points if point.valid is True and point.selected is True])
            if nselected > 0:
                print(f"Removing {nselected} points at a threshold of {init_thrsh}")
                f.removePoints(init_thrsh)
                init_thrsh -= thrsh_incr
                chunk.optimizeCameras(adaptive_fitting=True,fit_f = True, fit_cx = True, fit_cy = True, fit_b1 = True, fit_b2 = True, fit_k1 = True, fit_k2 = True, fit_k3 = True, fit_k4 = True, fit_p1 = True, fit_p2 = True)
                points = chunk.tie_points.points
                npoints = len(points)

In [ ]:
def grad_selection_PA(PA_thrsh, num_tries=4, pct_del=10, thrsh_incr=1):
    n=0
    target_thrsh = 3
    init_thrsh=PA_thrsh
    points = chunk.tie_points.points
    points_start_num = len(points)
    npoints = len(points)
    f = Metashape.TiePoints.Filter()
    f.init(chunk, criterion = Metashape.TiePoints.Filter.ProjectionAccuracy)
    f.selectPoints(init_thrsh)
    # count number of points selected
    nselected = len([True for point in points if point.valid is True and point.selected is True])
    # check to see if any points were selected
    pct_selected = (nselected / points_start_num)*100
    if init_thrsh<=target_thrsh and pct_selected <= 50: # job done
        print(f"Now removing {nselected} points at a threshold of {init_thrsh}")
        f.removePoints(init_thrsh)
        chunk.optimizeCameras(adaptive_fitting=True,fit_f = True, fit_cx = True, fit_cy = True, fit_b1 = True, fit_b2 = True, fit_k1 = True, fit_k2 = True, fit_k3 = True, fit_k4 = True, fit_p1 = True, fit_p2 = True)
    else:
        while True:
            n+=1
            if n>num_tries or init_thrsh<=target_thrsh or (100*((points_start_num-npoints)/points_start_num))>=50:
                break
            else:
                points = chunk.tie_points.points
                npoints = len(points)
                f.selectPoints(init_thrsh)
                nselected = len([True for point in points if point.valid is True and point.selected is True])
                pct_selected = (nselected / npoints)*100
                while True:
                    if pct_selected <= pct_del:
                        init_thrsh -= thrsh_incr/5
                        f.selectPoints(init_thrsh)
                        nselected = len([True for point in points if point.valid is True and point.selected is True])
                        pct_selected = (nselected/npoints)*100
                    else:
                        break
                f.selectPoints(init_thrsh)
                nselected = len([True for point in points if point.valid is True and point.selected is True])
                if nselected > 0:
                    print(f"Removing {nselected} points at a threshold of {init_thrsh}")
                    f.removePoints(init_thrsh)
                    init_thrsh -= thrsh_incr
                    chunk.optimizeCameras(adaptive_fitting=True,fit_f = True, fit_cx = True, fit_cy = True, fit_b1 = True, fit_b2 = True, fit_k1 = True, fit_k2 = True, fit_k3 = True, fit_k4 = True, fit_p1 = True, fit_p2 = True)
                    points = chunk.tie_points.points
                    npoints = len(points)

In [ ]:
def grad_selection_RE(RE_thrsh, num_tries=10, pct_del=10, thrsh_incr=0.1):
    n=0
    target_thrsh=0.3
    init_thrsh=RE_thrsh
    points = chunk.tie_points.points
    points_start_num = len(points)
    npoints = len(points)
    f = Metashape.TiePoints.Filter()
    f.init(chunk, criterion = Metashape.TiePoints.Filter.ReprojectionError)
    f.selectPoints(init_thrsh)
    # count number of points selected
    nselected = len([True for point in points if point.valid is True and point.selected is True])
    # check to see if any points were selected
    pct_selected = (nselected / points_start_num)*100
    if init_thrsh <= target_thrsh and pct_selected <= pct_del: # job done
        print(f"Now removing {nselected} points at a threshold of {init_thrsh}")
        f.removePoints(init_thrsh)
        chunk.optimizeCameras(adaptive_fitting=True,fit_f = True, fit_cx = True, fit_cy = True, fit_b1 = True, fit_b2 = True, fit_k1 = True, fit_k2 = True, fit_k3 = True, fit_k4 = True, fit_p1 = True, fit_p2 = True)
    else:
        while True:
            n+=1
            if n>num_tries or init_thrsh<=target_thrsh or (100*((points_start_num-npoints)/points_start_num))>=pct_del:
                break
            else:
                points = chunk.tie_points.points
                npoints = len(points)
                f.selectPoints(init_thrsh)
                nselected = len([True for point in points if point.valid is True and point.selected is True])
                pct_selected = (nselected / npoints)*100
                while True:
                    if pct_selected <= pct_del:
                        init_thrsh -= thrsh_incr
                        f.selectPoints(init_thrsh)
                        nselected = len([True for point in points if point.valid is True and point.selected is True])
                        pct_selected = (nselected/npoints)*100
                    else:
                        break
                f.selectPoints(init_thrsh)
                nselected = len([True for point in points if point.valid is True and point.selected is True])
                if nselected > 0:
                    print(f"Removing {nselected} points at a threshold of {init_thrsh}")
                    f.removePoints(init_thrsh)
                    init_thrsh -= thrsh_incr
                    chunk.optimizeCameras(adaptive_fitting=True,fit_f = True, fit_cx = True, fit_cy = True, fit_b1 = True, fit_b2 = True, fit_k1 = True, fit_k2 = True, fit_k3 = True, fit_k4 = True, fit_p1 = True, fit_p2 = True)
                    points = chunk.tie_points.points
                    npoints = len(points)

In [ ]:
# Filter tie points before reconstruction
RU_thrsh = 12
PA_thrsh = 6
RE_thrsh = 0.5

# reconstruction uncertainty
grad_selection_RU(RU_thrsh)
doc.save()

# projection accuracy
print(f"Now performing gradual selection using projection accuracy with a threshold of {PA_thrsh}")
grad_selection_PA(PA_thrsh)
doc.save()

# reprojection error
print(f"Now performing gradual selection using reprojection error with a threshold of {RE_thrsh}")
grad_selection_RE(RE_thrsh)
doc.save()

##### Build depth maps and point cloud

In [ ]:
# build depth maps
chunk.buildDepthMaps(downscale = 1, 
                     filter_mode = Metashape.MildFiltering,
                     reuse_depth = True)
doc.save()

In [5]:
# open doc from output folder not read_only mode
doc = Metashape.Document()
doc.open(output_folder + "project.psx", read_only=False, ignore_lock=True)
chunk = doc.chunk

In [ ]:
# filter dense cloud
point_cloud = doc.chunk.point_cloud
point_cloud.setConfidenceFilter(0, 1)
point_cloud.cropSelectedPoints()
point_cloud.setConfidenceFilter(0, 255)
point_cloud.compactPoints()
doc.save()

##### Check for transforms and apply if needed

In [ ]:
has_transform = chunk.transform.scale and chunk.transform.rotation and chunk.transform.translation

if has_transform:
    chunk.buildPointCloud(source_data=Metashape.DepthMaps, point_colors=True, point_confidence=True, keep_depth=True)
    doc.save()

    # filter dense cloud
    point_cloud = doc.chunk.point_cloud
    point_cloud.setConfidenceFilter(0, 1)
    point_cloud.cropSelectedPoints()
    point_cloud.setConfidenceFilter(0, 255)
    point_cloud.compactPoints()
    doc.save()

    chunk.buildDem(source_data=Metashape.PointCloud)
    doc.save()

    chunk.buildOrthomosaic(surface_data=Metashape.Model)
    doc.save()

##### Export processing outputs

In [ ]:
if chunk.point_cloud:
    chunk.exportPointCloud(output_folder + 'point_cloud.las', source_data = Metashape.PointCloud)

if chunk.elevation:
    chunk.exportRaster(output_folder + 'dem.tif', source_data = Metashape.Elevation)

if chunk.orthomosaic:
    chunk.exportRaster(output_folder + 'orthomosaic.tif', source_data = Metashape.Orthomosaic)

# export results
chunk.exportReport(output_folder + 'report.pdf')

print('Processing finished, results saved to ' + output_folder + '.')

In [ ]:
# Number of tie points
tie_points = chunk.point_cloud
print(f"Total tie points: {len(tie_points)}")

# Mean reprojection error
if len(tie_points) > 0:
    mean_reproj_error = sum(p.error for p in tie_points if p.valid) / len(tie_points)
    print(f"Mean reprojection error: {mean_reproj_error:.4f} pixels")

# Check how many cameras are aligned
aligned_cameras = sum(1 for cam in chunk.cameras if cam.transform is not None)
total_cameras = len(chunk.cameras)
print(f"Aligned Cameras: {aligned_cameras}/{total_cameras}")

# Print alignment success rate
alignment_rate = (aligned_cameras / total_cameras) * 100
print(f"Alignment Success Rate: {alignment_rate:.2f}%")

In [6]:
# open project 
doc = Metashape.Document()
doc.open("S:\\Zack\\Imagery\\Chestnut\\Agisoft\\2024\\Wintergreen\\AVNW\\Outputs_20250318_164146\\20250318_164146_project.psx", read_only=False, ignore_lock=True)  

In [7]:
current_time = "20250318_164146"
folder = "S:\\Zack\\Imagery\\Chestnut\\Agisoft\\2024\\Wintergreen\\AVNW\\Outputs_20250318_164146\\"

In [11]:
try:
    has_transform = (doc.chunk.transform.scale and doc.chunk.transform.rotation and doc.chunk.transform.translation)
    if has_transform:
        # # Build point cloud from depth maps
        # chunk.buildPointCloud(source_data=Metashape.DataSource.DepthMapsData, 
        #                         point_colors=True, 
        #                         point_confidence=True, 
        #                         keep_depth=True)
        # print("Point cloud finished building.")
        # chunk.point_cloud.setConfidenceFilter(0, 1)
        # chunk.point_cloud.cropSelectedPoints()
        # chunk.point_cloud.setConfidenceFilter(0, 255)
        # chunk.point_cloud.compactPoints()
        # print("Point cloud filtered.")
        # chunk.point_cloud.classifyGroundPoints(cell_size=0.25)
        # print("Point cloud ground points classified.")
        # doc.save()
        
        # # Build DEM, Model, and Orthomosaic
        # doc.chunk.buildDem(source_data=Metashape.DataSource.PointCloudData,
        #                 interpolation=Metashape.Interpolation.EnabledInterpolation)
        # print("DEM finished building.")
        # doc.save()
        
        doc.chunk.buildModel(source_data=Metashape.DataSource.DepthMapsData, 
                            surface_type=Metashape.SurfaceType.Arbitrary, 
                            interpolation=Metashape.Interpolation.EnabledInterpolation,
                            face_count=Metashape.FaceCount.HighFaceCount,
                            split_in_blocks=True,
                            workitem_size_cameras=15,
                            max_workgroup_size=75)
        print("Mesh finished building.")
        doc.save()
        
        doc.chunk.buildOrthomosaic(surface_data=Metashape.DataSource.ModelData, 
                                blending_mode=Metashape.BlendingMode.MosaicBlending,
                                ghosting_filter=True,
                                fill_holes=True,
                                cull_faces=False,
                                refine_seamlines=True)
        print("Orthomosaic finished building.")
        doc.save()
except Exception as e:
    print(f"Error during DEM/Model/Orthomosaic building for folder {folder}: {e}")

Error during DEM/Model/Orthomosaic building for folder S:\Zack\Imagery\Chestnut\Agisoft\2024\Wintergreen\AVNW\Outputs_20250318_164146\: cuCtxCreate failed: CUDA_ERROR_UNKNOWN (999) at line 196


In [ ]:
wgs84 = Metashape.CoordinateSystem("EPSG::4326")

<CoordinateSystem 'WGS 84 (EPSG::4326)'>

In [ ]:
try:
    # Export results
    if doc.chunk.point_cloud:
        pc_file = os.path.join(folder, f"{current_time}_point_cloud.las")
        doc.chunk.exportPointCloud(pc_file, source_data=Metashape.DataSource.PointCloudData, crs=wgs84)
        print("Point cloud exported.")

    if doc.chunk.elevation:
        dem_file = os.path.join(folder, f"{current_time}_dem.tif")
        doc.chunk.exportRaster(dem_file, source_data=Metashape.DataSource.ElevationData)
        print("DEM exported.")

    if doc.chunk.orthomosaic:
        ortho_file = os.path.join(folder, f"{current_time}_orthomosaic.tif")
        doc.chunk.exportRaster(ortho_file, source_data=Metashape.DataSource.OrthomosaicData)
        print("Orthomosaic exported.")

    report_file = os.path.join(folder, f"{current_time}_report.pdf")
    doc.chunk.exportReport(report_file)
    print("Report exported.")

    print(f"Processing finished for folder {folder}; results saved to {folder}.")
except Exception as e:
    print(f"Error during export for folder {folder}: {e}")